# Using CSBM generated data to train a model

In [6]:
import torch
import matplotlib.pyplot as plt

In [7]:
example_dataset = torch.load('./data/csbm/base_0.pt')
n = example_dataset[0].x.size()[0]
c = example_dataset[0].y.unique().size()[0]
d = example_dataset[0].x.size()[1]

In [8]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(d, 128)
        self.conv2 = GCNConv(128, c)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

## Retrain model for each task

In [9]:
from measures import Result

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.set_printoptions(precision=2)

for i in range(10):
    base_data = torch.load(f'./data/csbm/base_{i}.pt')
    zero_data = torch.load(f'./data/csbm/zero_{i}.pt')
    feat_data = torch.load(f'./data/csbm/feat_{i}.pt')
    struct_data = torch.load(f'./data/csbm/hom_{i}.pt')
    homophily_data = torch.load(f'./data/csbm/struct_{i}.pt')
    class_data = torch.load(f'./data/csbm/class_{i}.pt')

    
    base_result = Result(base_data, GCN().to(device))
    zero_result = Result(zero_data, GCN().to(device))
    feat_result = Result(feat_data, GCN().to(device))
    struct_result = Result(struct_data, GCN().to(device))
    homophily_result = Result(homophily_data, GCN().to(device))
    class_result = Result(class_data, GCN().to(device))
    
    results = [base_result, zero_result, feat_result, struct_result, homophily_result, class_result]
    names = {base_result: 'Base-CSBM',
             zero_result: 'Zero-CSBM',
             feat_result: 'Feature-CSBM',
             struct_result: 'Structure-CSBM',
             homophily_result: 'Homophily-CSBM',
             class_result: 'Class-CSBM'}
    for result in results:
        result.learn()
    
    for result in results:
        print('\n' + 10 * '=' + f' {names[result]} ' + 10 * '=')
        print(result.get_result_matrix())
        ap = result.get_average_accuracy()
        af = 0 if result == zero_result else result.get_average_forgetting_measure()
        print(f'\n AP: {ap:.2f}'.ljust(10) + f' AF: {af:.2f}')

cuda


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
